Why do we use average, not minimum or maximum?
- minimum would mean taking the weight of directed link from the animal with lower share, maximum with greater. Minimum means that we can safely assume that both of them are ???? TO do - 
- average - ok, fine.
I think taking the minimum could make sense because it guarantees that both species are similar of at least this bar to each other.
But for now we go with average



You project hosts onto parasites 
then, you want to answer if this projection was good?
Maybe it's OK?




In [45]:
import networkx as nx
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import pickle

In [54]:
df1 = pd.read_csv('./full_data.csv')
df1.columns
y = set(df1['Parasite'].unique())

In [55]:
df2 = pd.read_csv('./data/nodes.csv')
df2.columns

x = set(df2[df2[' is_host']==0][' name'])

In [64]:
x-y

{'Capillaria linsi',
 'Hymenolepis rangoonica',
 'Paragymnophallus kinsellai',
 'Longicollum indicum',
 'Camallanides dhamini',
 'Didymocystis kamegaii',
 'Loxogenoides bicolor',
 'Karachitrema trilobata',
 'Diplotriaena couturieri',
 'Echinostoma jurini',
 'Taenia polycalcaria',
 'Leidynemella fusiformis',
 'Pleurogenoides compactus',
 'Strigea elliptica',
 'Parasitodiplogaster yoponema',
 'Oochoristica trachysauri',
 'Physaloptera tigrinae',
 'Proteocephalus magna',
 'Erilepturus karachiensis',
 'Cercaria agarwali',
 'Magnacetabulum glandocaudum',
 'Pristotrema manteri',
 'Davainea capillaris',
 'Hystrignathus ferox',
 'Davainea macrocirrosa',
 'Glomericirrus propositus',
 'Endocotyle bushi',
 'Rhabdochona uca',
 'Rictularia jodhpurensis',
 'Paragonimus proliferus',
 'Bucephalus oari',
 'Pallisentis vietnamensis',
 'Cercaria srivastavi',
 'Amphibiocapillaria texensis',
 'Sonsinotrema calloti',
 'Porrocaecum pentkotai',
 'Camallanus tholukodensis',
 'Stegodexamene watsoni',
 'Paragoni

5870

Below I'm assigning every property to the host, such that after projection we loose no data.
Remember:
- animalis might be victims of multiple paragroups
- animals are assigned to one hostgroup.
- animals might be in multiple localities.
- parasites are assigned to one hostgroup and one paragroup and multiple localities

In [ ]:
#projection of parasites onto hosts
#1-Mode projection
from utils import obtain_bipartite_animals_parasites_graph

badj, x, h = obtain_bipartite_animals_parasites_graph(df)

KeyError: 'Adriatic Sea'

In [26]:
h.nodes(data=True)['Acanthocephaloides propinquus']

{'bipartite': 1,
 'host_group': {'Chondrichthyes', 'Teleostei'},
 'para_group': 'Acanthocephala',
 'locality': {'Adriatic Sea',
  'Atlantic Ocean',
  'Azov Sea',
  'Black Sea',
  'France',
  'Northeast Atlantic',
  'Sea of Marmara',
  'Spain + Andalusia',
  'Western Europe',
  'Western Mediterranean'}}

In [ ]:


bipartite_1_nodes = [n for n, data in G.nodes(data=True) if data.get('bipartite') == 1]

print(len(bipartite_1_nodes), bipartite_1_nodes)

projected_G = bipartite.projected_graph(G, bipartite_1_nodes)

print(len(projected_G.nodes(data=True)) ,len(projected_G.edges(data=True)))

# Copy attributes from the original graph G to projected_G
for node in projected_G.nodes():
    if node in G.nodes():
        # Copy all attributes of the node from the original graph
        projected_G.nodes[node].update(G.nodes[node])

# Save the graph to a file
with open('projected_graph_para.gpickle', 'wb') as f:
    pickle.dump(projected_G, f, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# Projection of parasites onto  (hosts remain)
G = nx.Graph()

for i, el in df.iterrows():
    if G.has_node(el['Parasite']):
        G.add_node(el['Parasite'], bipartite=0)
    if G.has_node(el['Host']):
        G.nodes[el['Host']]['locality'].add(el['locality'])
        G.nodes[el['Host']]['para_group'].add(el['group'])
    else:
        locality = set()
        locality.add(el['locality'])
        para_group = set()
        para_group.add(el['group'])
        G.add_node(el['Host'], bipartite=1, host_group=el['hostgroup'], para_group=para_group, locality=locality)
    if not G.has_edge(el['Host'], el['Parasite']):
        G.add_edge(el['Host'], el['Parasite'], )

In [18]:
list(df.iterrows())[2]

(2,
 Unnamed: 0.1                                                    2
 Unnamed: 0                                                      4
 Host                                            Gobius cruentatus
 Parasite                            Acanthocephaloides propinquus
 ParasiteFull       Acanthocephaloides propinquus (Dujardin, 1845)
 group                                              Acanthocephala
 locality                                             Adriatic Sea
 hostgroup                                               Teleostei
 animal_nodes                                                 1074
 parasites_nodes                                             15711
 Name: 2, dtype: object)

In [19]:
G.nodes(data=True)['Acanthocephaloides propinquus']

{'bipartite': 0}

In [ ]:
max(dict(projected_G.degree()).values())

In [ ]:
def stats(graph):
    graph_degrees=dict(graph.degree()).values()
    stats_dict={}
    #average degree
    stats_dict["average_degree"]=sum(graph_degrees)/graph.number_of_nodes()
    #max degree
    stats_dict["max_degree"]=max(graph_degrees)
    #min degree
    stats_dict["min_degree"]=min(graph_degrees)
    #percentage under the average
    stats_dict["percentage_under_average"]=len([d for d in graph_degrees if d < stats_dict["average_degree"]])/len(graph_degrees)
    #assortativity
    stats_dict["degree_assortativity"]=nx.degree_assortativity_coefficient(graph)
    #modularity
    communities = nx.community.greedy_modularity_communities(graph)
    stats_dict["modularity"]=nx.community.modularity(graph,communities)
    #...
    return stats_dict


In [ ]:
s=stats(projected_G)

In [ ]:
import json
with open('stats/1-mode projection.txt', 'w+') as convert_file: 
     convert_file.write(json.dumps(s))

In [ ]:
# now we have directional weights, we need to get the average
from utils import average_resource_allocation
from utils import obtain_bipartite_animals_parasites_graph

badj, names = obtain_bipartite_animals_parasites_graph(df)


In [22]:
np.array([0])/np.int16(0)

C:\Users\Maciej\AppData\Local\Temp\ipykernel_28604\302908645.py:1: RuntimeWarning: invalid value encountered in divide
  np.array([0])/np.int16(0)


array([nan])

In [23]:

A = badj / badj.sum(axis=1)[:, np.newaxis]

C:\Users\Maciej\AppData\Local\Temp\ipykernel_28604\3967807711.py:1: RuntimeWarning: invalid value encountered in divide
  A = badj / badj.sum(axis=1)[:, np.newaxis]


In [ ]:
resource_allocation_weights.

array([[0.06535948, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.26136364, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.11666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05631868, 0.03846154,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03846154, 0.28846154,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [32]:
np.sum(resource_allocation_weights - resource_allocation_weights.T)

np.float64(0.0)

In [29]:
?average_resource_allocation

Signature: average_resource_allocation(badj: <built-in function array>) -> <built-in function array>
Docstring: ddd
File:      c:\users\maciej\projects\parasites_network_analysis\resource_allocation.py
Type:      function

In [ ]:

resource_allocation_weights = average_resource_allocation(badj)



# sanity check: symmetric matrix
print(np.allclose(resource_allocation_weights, resource_allocation_weights.T))
# compute the number of edges
print(np.count_nonzero(~np.isclose(resource_allocation_weights,0)) / 2)


True
528861.0


In [ ]:
sum(resource_allocation_weights[1, :])

SELF LOOPS, are we care about them? not much tbh, we can remove them later.
We don't care - we need to remove them later.

In [ ]:

# are there any nans in resource_allocation_weights?
print(np.isnan(resource_allocation_weights).any())
# are there any inf in resource_allocation_weights?
print(np.isinf(resource_allocation_weights).any())
# are there any rows that sum to zero? (contains only zeros)
print(np.any(np.isclose(resource_allocation_weights.sum(axis=1), 0)))
# are there any rows which have only one non-zero element?
print(np.any(np.count_nonzero(resource_allocation_weights, axis=1) == 1))

# are there rows not summing to 1?
print(np.any(np.isclose(resource_allocation_weights.sum(axis=1), 1) == False))

In [ ]:
# do the noise corrected disparity filter
# You don't need to take care of zeros, because these are going to be zero anyway after the filtering


node_edges_weights_sum = resource_allocation_weights.sum(axis=1)
edge_neighbourhood_weights_sums = node_edges_weights_sum[:, np.newaxis] - resource_allocation_weights
# are there rows that sum to zero? (contains only zeros)
print(np.any(np.isclose(edge_neighbourhood_weights_sums.sum(axis=1), 0)))

edge_neighbourhood_weights_means = edge_neighbourhood_weights_sums / (np.count_nonzero(resource_allocation_weights, axis=1) - 1)[:, np.newaxis]
# there could be 0/0, which would result in nan, replace these with zeros
# I don't understand how infs emerge, but they do, so replace them with zeros
edge_neighbourhood_weights_means = np.nan_to_num(edge_neighbourhood_weights_means, nan=0, posinf=0, neginf=0)

# above should not be symmetric
print(np.allclose(edge_neighbourhood_weights_means, edge_neighbourhood_weights_means.T))
disparity_filter = (edge_neighbourhood_weights_means + edge_neighbourhood_weights_means.T) / 2

# where the values are below the disparity filter, set to zero
filtered_weights = resource_allocation_weights.copy()
# check if both disparity filter and resource_allocation_weights are symmetric
print(np.allclose(disparity_filter, disparity_filter.T))
print(np.allclose(resource_allocation_weights, resource_allocation_weights.T))
filtered_weights[resource_allocation_weights < disparity_filter] = 0
filtered_weights

In [ ]:

# check if the filtered weights are symmetric
print(np.allclose(filtered_weights, filtered_weights.T))
# compute the number of values which are not close to zero
print(np.count_nonzero(~np.isclose(filtered_weights, 0))/2)
filtered_weights.shape

Filtered weights is not symmetric, but has inconsistent amount of edges? :()

In [ ]:
# create a new undirected graph with the filtered weights as adjacency matrix
G_filtered = nx.from_numpy_array(filtered_weights)
# reassign the node names and attributes
G_filtered = nx.relabel_nodes(G_filtered, dict(enumerate(bipartite_one_nodes)))
# add the attributes
for n, d in G.nodes(data=True):
    if d.get('bipartite') == 1:
        # drop the bipartite attribute
        d.pop('bipartite')
        G_filtered.nodes[n].update(d)